# Processing EK60 Data to Extract Target Strength


## Step 1: Fetch Configuration Files

We begin by importing the required libraries and specifying the paths for the dataset and pipeline configuration files. These files contain the necessary information for data processing.

In [1]:
from pathlib import Path
from echodataflow import echodataflow_start, glob_url

dataset_config = Path("./datastore.yaml").resolve()
pipeline_config = Path("./pipeline.yaml").resolve()

### Step 1.1: Configuration Files

Familiarize yourself with the configuration options by exploring the documentation for:

- Pipeline Configuration: Learn about configuration settings for the pipeline by referring to [Pipeline Configuration](./pipelineconfiguration.md).

- Datastore Configuration: Understand the various configuration options related to data storage by reading [Datastore Configuration](./datastoreconfiguration.md).

These documents provide detailed information on the configurations used during the setup process.

## Step 2: Getting Data
Next, we'll use the glob_url function to retrieve a list of URLs matching a specific pattern. In this case, we're targeting raw EK60 data files from the SH1707 survey.

In [2]:
all_files = glob_url("s3://ncei-wcsd-archive/data/raw/Bell_M._Shimada/SH1707/EK60/*.raw", {'anon':True})

c:\Users\soham\anaconda3\envs\echodataflowmem\lib\site-packages\fsspec\registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


## Step 3: Preparing Files
We'll now extract the file names from the URLs and create a file listing for the transect. This will help us organize and work with the data effectively.

In [3]:
files = []
for file in all_files:
    f = file.split(".r")[0]
    files.append(f.split("/")[-1])

transect = open('EK60_SH1707_Shimada.txt','w')
i = 0
for f in files:
    if i == 20:
        break
    transect.write(f+".raw\n")
    i = i + 1
transect.close()

## Step 4: Processing with echodataflow
Now, we're ready to kick off the data processing using echodataflow. We'll provide the dataset and pipeline configurations, along with additional options.

In [4]:
options = {"storage_options_override": False}
data  = echodataflow_start(dataset_config=dataset_config, pipeline_config=pipeline_config, options=options)


Checking Configuration

Configuration Check Completed

Checking Connection to Prefect Server

Starting the Pipeline
echodataflow_trigger.py  Echodataflow Trigger : Dataset Configuration Loaded For This Run
echodataflow_trigger.py  Echodataflow Trigger : --------------------------------------------------
echodataflow_trigger.py  Echodataflow Trigger : {"name": "Bell_M._Shimada-SH1707-EK60", "sonar_model": "EK60", "raw_regex": "(.*)-?D(?P<date>\\w{1,8})-T(?P<time>\\w{1,6})", "args": {"urlpath": "s3://ncei-wcsd-archive/data/raw/{{ ship_name }}/{{ survey_name }}/{{ sonar_model }}/*.raw", "parameters": {"ship_name": "Bell_M._Shimada", "survey_name": "SH1707", "sonar_model": "EK60"}, "storage_options": {"anon": true}, "group": {"file": "./EK60_SH1707_Shimada.txt"}, "group_name": 2017, "json_export": true}, "output": {"urlpath": "./echodataflow-output", "retention": true, "overwrite": true}}
{'name': 'Bell_M._Shimada-SH1707-EK60', 'sonar_model': 'EK60', 'raw_regex': '(.*)-?D(?P<date>\\w{1,8}

c:\Users\soham\anaconda3\envs\echodataflowmem\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64184 instead
  warnings.warn(


{'mod_name': 'initialization_flow.py', 'func_name': 'Init Flow'} : <Client: 'tcp://127.0.0.1:64186' processes=5 threads=20, memory=15.63 GiB>
initialization_flow.py  Init Flow : <Client: 'tcp://127.0.0.1:64186' processes=5 threads=20, memory=15.63 GiB>
{'mod_name': 'initialization_flow.py', 'func_name': 'Init Flow'} : Scheduler at : tcp://127.0.0.1:64186
initialization_flow.py  Init Flow : Scheduler at : tcp://127.0.0.1:64186
{'mod_name': 'initialization_flow.py', 'func_name': 'Init Flow'} : --------------------------------------------------
initialization_flow.py  Init Flow : --------------------------------------------------
{'mod_name': 'initialization_flow.py', 'func_name': 'Init Flow'} : Executing stage : name='echodataflow_open_raw' module='echodataflow.stages.subflows.open_raw' external_params=None options={'save_raw_file': True, 'use_raw_offline': True, 'use_offline': True, 'use_dask': True} prefect_config=None dependson=None
initialization_flow.py  Init Flow : Executing stage 

## Step 5: Results
Finally, let's take a look at the first entry from the processed data.

In [5]:
data[0][0]

<xarray.Dataset> Size: 4MB
Dimensions:                        (channel: 3, ping_time: 19,
                                    range_sample: 3888, filenames: 1)
Coordinates:
  * channel                        (channel) <U37 444B 'GPT  18 kHz 009072058...
  * filenames                      (filenames) int32 4B 0
  * ping_time                      (ping_time) datetime64[ns] 152B 2017-06-15...
  * range_sample                   (range_sample) int32 16kB 0 1 2 ... 3886 3887
Data variables: (12/16)
    TS                             (channel, ping_time, range_sample) float64 2MB dask.array<chunksize=(2, 10, 1944), meta=np.ndarray>
    angle_offset_alongship         (channel) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    angle_offset_athwartship       (channel) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    angle_sensitivity_alongship    (channel) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    angle_sensitivity_athwartship  (channel) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    beamwidth_alongship            (channel) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    ...                             ...
    gain_correction                (channel, ping_time) float64 456B dask.array<chunksize=(3, 19), meta=np.ndarray>
    sa_correction                  (channel, ping_time) float64 456B dask.array<chunksize=(3, 19), meta=np.ndarray>
    sound_absorption               (channel, ping_time) float64 456B dask.array<chunksize=(3, 19), meta=np.ndarray>
    sound_speed                    (channel, ping_time) float64 456B dask.array<chunksize=(3, 19), meta=np.ndarray>
    source_filenames               (filenames) <U153 612B dask.array<chunksize=(1,), meta=np.ndarray>
    water_level                    float64 8B ...
Attributes:
    processing_function:          calibrate.compute_TS
    processing_software_name:     echopype
    processing_software_version:  0.8.4
    processing_time:              2024-05-24T23:50:09Z

**Congratulations!** You've successfully processed EK60 data using echodataflow. This notebook provides a simplified overview, and you can explore the capabilities of echodataflow for more advanced processing tasks.

Feel free to modify the parameters, paths, and configurations as needed to adapt to your data and requirements.